In [1]:
# STANDARD
from pprint import pprint

# PyPI
from bs4 import BeautifulSoup as BS
import requests
from tqdm import tqdm

# LOCAL
from interface_db import db_interface_sqlite as data

In [15]:
def parse_html(url: str) -> str:
    response = requests.get(url)
    return BS(response.content, 'html.parser')

def check_url_new(url: str) -> bool:
    with data.database() as db:
        if db.execute(f'SELECT url FROM categories WHERE url = \'{url}\'') == url:
            return False
        else:
            return True

In [16]:
def fetch_and_insert_categories(website_name: str) -> None:
    with data.database() as db:
        base = db.query_websites(website_name)
        print(f'Pulling categories for {base.name}')
        parsed_html = parse_html(base.url)
        for item in parsed_html.find_all('div', attrs={"class":"related-content clearfix related-content-sm decorated channel-list"}):
            category_url = item.find('a')['href']
            category_name = category_url.rsplit("/")[3]
            category = data.category(category_name, category_url, base.id)
            print(category.url)
            if check_url_new(category.url) == True:
                db.insert_category(category)
                print('inserted')
            else:
                print('not inserted')



fetch_and_insert_categories('Patheos Blogs')



Pulling categories for Patheos Blogs
https://www.patheos.com/buddhist-blogs
inserted
https://www.patheos.com/catholic-blogs
inserted
https://www.patheos.com/contemplative-blogs
inserted
https://www.patheos.com/evangelical-blogs
inserted
https://www.patheos.com/general-christian-blogs
inserted
https://www.patheos.com/hindu-blogs
inserted
https://www.patheos.com/jewish-blogs
inserted
https://www.patheos.com/latter-day-saint-blogs
inserted
https://www.patheos.com/muslim-blogs
inserted
https://www.patheos.com/new-visions-blogs
inserted
https://www.patheos.com/nonreligious-blogs
inserted
https://www.patheos.com/pagan-blogs
inserted
https://www.patheos.com/progressive-christian-blogs
inserted
https://www.patheos.com/more-voices-blogs
inserted
https://www.patheos.com/sixseeds-family-blogs
inserted
https://www.patheos.com/politics-blue-blogs
inserted
https://www.patheos.com/politics-red-blogs
inserted
https://www.patheos.com/entertainment-blogs
inserted
https://www.patheos.com/faith-and-work-b

In [42]:
def fetch_blogs():
    """[ ] Document for refactoring
    
    """
    url = 'http://www.patheos.com/blogs'
    response = requests.get(url)
    soup = BS(response.content, 'html.parser')

    blog_list = []

    for blog in soup.find_all('div', attrs={"class":"related-content clearfix related-content-sm decorated channel-list"}):
        blog_url1 = blog.find('a')
        blog_url_test = blog.find('a')['href']
        blog_url2 = blog_url1['href']
        blog_list.append(blog_url2) 
    
    print('blog_url2')
    pprint(blog_url_test)
    pprint(blog_url1)
    pprint(blog_url2)
    pprint(blog_list)

    blog_lists = []
    
    for i in blog_list:
        split_url = i.rsplit("/")
        blog_name = split_url[3]
        blog_lists.append(blog_name)
    
    blog_dict = {i:[] for i in blog_lists}

    blog_cat_prefix = "https://www.patheos.com/"

    i = 0    
    for blog_urls in tqdm(blog_dict, desc='Fetch blog urls for each category'):
        if blog_urls:
            query_url = blog_cat_prefix + blog_urls
            #sleep(random.uniform(1, 3))
            subsub_blog = requests.get(query_url)
            soup2 = BS(subsub_blog.content, 'html.parser')
            for blog in soup2.find_all('div', attrs={"class":"author-info"}):
                for blog_url0 in blog.find_all('div', attrs={"class":"title"}):
                    blog_url1 = blog_url0.find('a')
                    blog_url2 = blog_url1['href']
                    blog_dict[blog_urls].append(blog_url2)

            i = i + 1
        else:
            continue
    return blog_dict

In [43]:
results = fetch_blogs()

Fetch blog urls for each category:   0%|          | 0/20 [00:00<?, ?it/s]blog_url2
'https://www.patheos.com/patheos-partner-blogs'
<a href="https://www.patheos.com/patheos-partner-blogs"><img alt="" class="lazyblur lazyload" data-src="https://media.patheos.com/~/media/images/components/channel-list/icon-partner.svg?as=1&amp;h=70&amp;w=70" src="https://media.patheos.com/~/media/images/components/channel-list/icon-partner.svg?as=1&amp;h=70&amp;w=70?w=25"/></a>
'https://www.patheos.com/patheos-partner-blogs'
['https://www.patheos.com/buddhist-blogs',
 'https://www.patheos.com/catholic-blogs',
 'https://www.patheos.com/contemplative-blogs',
 'https://www.patheos.com/evangelical-blogs',
 'https://www.patheos.com/general-christian-blogs',
 'https://www.patheos.com/hindu-blogs',
 'https://www.patheos.com/jewish-blogs',
 'https://www.patheos.com/latter-day-saint-blogs',
 'https://www.patheos.com/muslim-blogs',
 'https://www.patheos.com/new-visions-blogs',
 'https://www.patheos.com/nonreligious

In [25]:
pprint(results)

://www.patheos.com/blogs/abletochoose',
                        'https://www.patheos.com/blogs/accordingtomatthew',
                        'https://www.patheos.com/blogs/rivers',
                        'https://www.patheos.com/blogs/anotherbreedoffaith',
                        'https://www.patheos.com/blogs/anotherwhiteatheistincolombia',
                        'https://www.patheos.com/blogs/theatheistandthepastor',
                        'https://www.patheos.com/blogs/atheistallie',
                        'https://www.patheos.com/blogs/theatheistrabbi',
                        'https://www.patheos.com/blogs/atheology',
                        'https://www.patheos.com/blogs/barrierbreaker',
                        'https://www.patheos.com/blogs/blamelesssky',
                        'https://www.patheos.com/blogs/blinkyandsal',
                        'https://www.patheos.com/blogs/brotherrichard',
                        'https://www.patheos.com/blogs/camelswithhammers',
       

In [26]:
cat = 0
blog = 0

for k in results:
    cat += 1
    for i in results[k]:
        blog += 1

print(cat)
print(blog)

20
597
